In [14]:
!pip install tensorflow-io


In [15]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import polars as pl
import tensorflow_io as tfio
from PIL import Image
import numpy as np
import io

load_dotenv()

True

In [16]:
def load_and_convert_image(filepath: str):
    full_path = os.path.join(os.environ["PATH_ARCHIVE"], filepath.replace("/", os.sep))
    im = Image.open(full_path)
    return np.array(im).tobytes()


def load_and_convert_image_2(filepath: str):
    full_path = os.path.join(os.environ["PATH_ARCHIVE"], filepath.replace("/", os.sep))
    im = Image.open(full_path)
    image_bytes = io.BytesIO()
    im.save(image_bytes, format="JPEG")
    return image_bytes.getvalue()


def clean_df(df: pl.DataFrame):
    df = df.with_columns(pl.col("image:FILE").str.split(by="/").alias("split"))
    df = df.select(
        pl.col("split").list.get(2).str.strip_suffix("_c.jpg").alias("id"),
        pl.col("split").list.get(1).alias("flower_type"),
        pl.col("category"),
        pl.col("image:FILE")
        .map_elements(load_and_convert_image_2, return_dtype=pl.Binary)
        .alias("data"),
        pl.col("image:FILE").alias("filepath"),
    )
    filepath = df.row(0, named=True)["filepath"]
    print(len(load_and_convert_image(filepath)))
    print(len(load_and_convert_image_2(filepath)))
    print(len(df.row(0, named=True)["data"]))
    return df


def upload_to_collection(collection, csv_filename, reset=True):
    df = pl.read_csv(os.path.join(os.environ["PATH_ARCHIVE"], csv_filename))

    if reset:
        result = collection.delete_many({})
        print(result.deleted_count)

    # remove the paths from the df
    df = clean_df(df)

    # insert into mongodb
    result = collection.insert_many(df.to_dicts())
    document_ids = result.inserted_ids
    print("# of documents inserted: " + str(len(document_ids)))
    print(f"_ids of inserted documents: {document_ids}")

In [17]:
# setup database
client = MongoClient(os.environ["URI"])

# Get reference to 'bank' database
db = client.flowers

# Get a reference to 'magazines' collection
train_collection = db.train

In [20]:
upload_to_collection(train_collection, "train.csv")

196608
12108
12108


In [23]:
client.close()